In [1]:
import numpy as np
from scipy import stats
from numpy import mean
from numpy.random import beta, poisson
from scipy.special import j_roots
from scipy.special import beta as beta_fun
from matplotlib import pyplot as plt
import pandas as pd
import anndata as ad

import warnings
warnings.filterwarnings('ignore')

In [2]:
adata = ad.read('../data/processed/mus_musculus_preprocessed.h5ad')

In [3]:
cells = len(adata.obs.index)
cells

682

In [4]:
# moment-based inference
def MomentInference(vals, export_moments=False):
    # code from Anton Larsson's R implementation
    from scipy import stats # needs imports inside function when run in ipyparallel
    import numpy as np
    m1 = float(np.mean(vals))
    m2 = float(sum(vals*(vals - 1))/len(vals))
    m3 = float(sum(vals*(vals - 1)*(vals - 2))/len(vals))
    
    # sanity check on input (e.g. need at least on expression level)
    if sum(vals) == 0: return np.nan
    if m1 == 0: return np.nan
    if m2 == 0: return np.nan
    
    r1=m1
    r2=m2/m1
    r3=m3/m2
    
    if (r1*r2-2*r1*r3 + r2*r3) == 0: return np.nan
    if ((r1*r2 - 2*r1*r3 + r2*r3)*(r1-2*r2+r3)) == 0: return np.nan
    if (r1 - 2*r2 + r3) == 0: return np.nan
        
    lambda_est = (2*r1*(r3-r2))/(r1*r2-2*r1*r3 + r2*r3)
    mu_est = (2*(r3-r2)*(r1-r3)*(r2-r1))/((r1*r2 - 2*r1*r3 + r2*r3)*(r1-2*r2+r3))
    v_est = (2*r1*r3 - r1*r2 - r2*r3)/(r1 - 2*r2 + r3)
    
    if export_moments:
        return np.array([lambda_est, mu_est, v_est, r1, r2, r3])
    
    return np.array([lambda_est, mu_est, v_est])
def MaximumLikelihood(vals, export_asymp_ci = False, fix = 0, export_fun = False):
    from scipy.interpolate import interp1d
    from scipy.optimize import minimize
    from scipy import special
    from scipy.stats import poisson,norm
    from scipy.special import j_roots
    from scipy.special import beta as beta_fun    
    import numpy as np
    if len(vals) == 0:
        return np.array([np.nan, np.nan, np.nan])
    def dBP(at, alpha, bet, lam):
        at.shape = (len(at), 1)
        np.repeat(at, 50, axis = 1)
        def fun(at, m):
            if(max(m) < 1e6):
                return(poisson.pmf(at,m))
            else:
                return(norm.pdf(at,loc=m,scale=sqrt(m)))
        
        x,w = j_roots(50,alpha = bet - 1, beta = alpha - 1)
        gs = np.sum(w*fun(at, m = lam*(1+x)/2), axis=1)
        prob = 1/beta_fun(alpha, bet)*2**(-alpha-bet+1)*gs
        return(prob)
    def LogLikelihood(x, vals):
        kon = x[0]
        koff = x[1]
        ksyn = x[2]
        return(-np.sum(np.log( dBP(vals,kon,koff,ksyn) + 1e-10) ) )
    x0 = MomentInference(vals)
    if np.isnan(x0).any():
        x0 = np.array([10,10,10])
    bnds = ((1e-3,1e3),(1e-3,1e3), (1, 1e10))
    vals_ = np.copy(vals) # Otherwise the structure is violated.
    try:
        ll = minimize(LogLikelihood, x0, args = (vals_), method='L-BFGS-B', bounds = bnds)
    except:
        if export_fun:
            return np.array([np.nan,np.nan,np.nan]), np.nan
        return np.array([np.nan,np.nan,np.nan])
    #se = ll.hess_inv.todense().diagonal()
    if export_fun:
        return ll.x, ll.fun
    estim = ll.x
    return estim

In [5]:
def dBP(at, alpha, bet, lam):
    at.shape = (len(at),1)
    np.repeat(at, 50, axis = 1)
    def fun(at, m):
        if(max(m) < 1e6):
            return(stats.poisson.pmf(at,m))
        else:
            return(stats.norm.pdf(at,loc=m,scale=sqrt(m)))
    if alpha <= 0 or bet <= 0:
        return np.nan
    x,w = j_roots(50,alpha = bet-1, beta = alpha - 1)
    gs = np.sum(w*fun(at, m = lam*(1+x)/2), axis=1)
    prob = 1/beta_fun(alpha, bet)*2**(-alpha-bet+1)*gs
    return(prob)

In [6]:
c57 = adata.to_df('allele_c57').transpose()
cast = adata.to_df('allele_cast').transpose()

gene_df = pd.DataFrame(index=adata.var.index)

In [7]:
gene_df['c57_kon'] = 0.0
gene_df['c57_koff'] = 0.0
gene_df['c57_ksyn'] = 0.0
gene_df['cast_kon'] = 0.0
gene_df['cast_koff'] = 0.0
gene_df['cast_ksyn'] = 0.0
gene_df['total_kon'] = 0.0
gene_df['total_koff'] = 0.0
gene_df['total_ksyn'] = 0.0

In [8]:
# from joblib import Parallel, delayed

counter = 1

for gene in adata.var.index:
    mat_c57 = c57.loc[gene][pd.notnull(c57.loc[gene])]
    pat_cast = cast.loc[gene][pd.notnull(cast.loc[gene])]
    tot_kon_c57, tot_koff_c57, tot_ksyn_c57 = MaximumLikelihood(mat_c57)
    tot_kon_cast, tot_koff_cast, tot_ksyn_cast = MaximumLikelihood(pat_cast)  
    tot_kon, tot_koff, tot_ksyn = MaximumLikelihood(mat_c57+pat_cast)
    
    gene_df.loc[gene]['c57_kon'] = tot_kon_c57
    gene_df.loc[gene]['c57_koff'] = tot_koff_c57
    gene_df.loc[gene]['c57_ksyn'] = tot_ksyn_c57
    gene_df.loc[gene]['cast_kon'] = tot_kon_cast
    gene_df.loc[gene]['cast_koff'] = tot_koff_cast
    gene_df.loc[gene]['cast_ksyn'] = tot_ksyn_cast
    gene_df.loc[gene]['total_kon'] = tot_kon
    gene_df.loc[gene]['total_koff'] = tot_koff
    gene_df.loc[gene]['total_ksyn'] = tot_ksyn
    
#     print(counter, end=" ")
    
    counter+=1
    
#     if counter > 10:
#         break


# def parallel_calc(gene, c57, cast, gene_df):
#     mat_c57 = c57.loc[gene][pd.notnull(c57.loc[gene])]
#     pat_cast = cast.loc[gene][pd.notnull(cast.loc[gene])]
#     tot_kon_c57, tot_koff_c57, tot_ksyn_c57 = MaximumLikelihood(mat_c57)
#     tot_kon_cast, tot_koff_cast, tot_ksyn_cast = MaximumLikelihood(pat_cast)  
#     tot_kon, tot_koff, tot_ksyn = MaximumLikelihood(mat_c57+pat_cast)

#     gene_df.loc[gene]['c57_kon'] = tot_kon_c57
#     gene_df.loc[gene]['c57_koff'] = tot_koff_c57
#     gene_df.loc[gene]['c57_ksyn'] = tot_ksyn_c57
#     gene_df.loc[gene]['cast_kon'] = tot_kon_cast
#     gene_df.loc[gene]['cast_koff'] = tot_koff_cast
#     gene_df.loc[gene]['cast_ksyn'] = tot_ksyn_cast
#     gene_df.loc[gene]['total_kon'] = tot_kon
#     gene_df.loc[gene]['total_koff'] = tot_koff
#     gene_df.loc[gene]['total_ksyn'] = tot_ksyn
    
#     print('asdf', end="  ")
    
#     return gene_df


In [ ]:
# gene_df = Parallel(n_jobs=2)(delayed(parallel_calc)(gene, c57, cast, gene_df) for gene in ['Mrpl15', '4732440D04Rik', 'Cops5', 'Arfgef1', 'Tram1'])

In [9]:
gene_df.to_csv('../data/temp_params.csv')

In [10]:
gene_df

,c57_kon,c57_koff,c57_ksyn,cast_kon,cast_koff,cast_ksyn,total_kon,total_koff,total_ksyn
Gene,,,,,,,,,
Mrpl15,0.137540,1.025298,16.142093,0.122347,1.224672,19.529545,0.175476,1.132123,27.833175
4732440D04Rik,0.779018,12.361265,37.523179,0.699138,58.325099,158.448099,1.358319,403.981764,1222.432088
Cops5,0.784568,2.727551,56.500928,1.147452,72.681626,795.075994,2.557655,836.126843,8235.446613
Arfgef1,0.562565,4.123210,24.559485,0.575572,3.908667,23.000236,1.560758,255.002303,972.697642
Tram1,3.433453,448.080240,1451.860539,3.621442,472.175264,1350.464250,4.037842,694.491005,3705.471848
...,...,...,...,...,...,...,...,...,...
Exosc7,0.638748,113.483108,957.701819,0.615424,86.296659,732.917759,1.348519,221.798252,1742.364248
Lars2,0.110824,0.385575,3.631488,0.447942,1.585246,7.474467,0.978933,2.665669,9.151741
Sacm1l,0.431688,118.690680,593.689143,0.405336,147.228885,680.114701,0.573285,123.600184,917.878001


In [11]:
bdata = adata

In [12]:
temp = gene_df

In [13]:
adata.var = adata.var.join(gene_df)

In [14]:
adata.var

,Accession,Chromosome,End,Start,Strand,sum_allele_c57,sum_allele_cast,sum_ratio_allele_c57,sum_ratio_allele_cast,ratio_sum_allele_c57,...,Ribosomal_prot,c57_kon,c57_koff,c57_ksyn,cast_kon,cast_koff,cast_ksyn,total_kon,total_koff,total_ksyn
Gene,,,,,,,,,,,,,,,,,,,,,
Mrpl15,ENSMUSG00000033845,1,4785739,4773206,-,1175.0,1092.0,0.518306,0.481694,153.475673,...,other protein,0.137540,1.025298,16.142093,0.122347,1.224672,19.529545,0.175476,1.132123,27.833175
4732440D04Rik,ENSMUSG00000090031,1,6214590,6213293,-,1498.0,1261.0,0.542950,0.457050,308.882750,...,other protein,0.779018,12.361265,37.523179,0.699138,58.325099,158.448099,1.358319,403.981764,1222.432088
Cops5,ENSMUSG00000025917,1,10038127,10024602,-,8192.0,7966.0,0.506993,0.493007,323.482196,...,other protein,0.784568,2.727551,56.500928,1.147452,72.681626,795.075994,2.557655,836.126843,8235.446613
Arfgef1,ENSMUSG00000067851,1,10232670,10137571,-,1841.0,1842.0,0.499864,0.500136,284.025982,...,other protein,0.562565,4.123210,24.559485,0.575572,3.908667,23.000236,1.560758,255.002303,972.697642
Tram1,ENSMUSG00000025935,1,13589864,13564702,-,7514.0,7028.0,0.516710,0.483290,350.605317,...,other protein,3.433453,448.080240,1451.860539,3.621442,472.175264,1350.464250,4.037842,694.491005,3705.471848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Exosc7,ENSMUSG00000025785,9,123136129,123113215,+,3556.0,3447.0,0.507782,0.492218,317.695221,...,other protein,0.638748,113.483108,957.701819,0.615424,86.296659,732.917759,1.348519,221.798252,1742.364248
Lars2,ENSMUSG00000035202,9,123462664,123366940,+,376.0,763.0,0.330114,0.669886,114.946627,...,other protein,0.110824,0.385575,3.631488,0.447942,1.585246,7.474467,0.978933,2.665669,9.151741
Sacm1l,ENSMUSG00000025240,9,123592598,123529882,+,1444.0,1262.0,0.533629,0.466371,267.715853,...,other protein,0.431688,118.690680,593.689143,0.405336,147.228885,680.114701,0.573285,123.600184,917.878001


In [15]:
bdata.write('../data/processed/mus_musculus_preprocessed_with_params.h5ad')